<a href="https://colab.research.google.com/github/atrbyg24/gpt2-rlhf/blob/main/SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Supervised Fine-Tuning**

Supervised Fine-Tuning (SFT) is the first step in the entire RLHF fine-tuning pipeline (see Figure 2 in [RLHF paper](https://arxiv.org/pdf/2203.02155)). This notebook will use gpt2 and the corresponding tokenizer model from Hugging Face transformers library to perform SFT on stanfordnlp/sst2 dataset.

**Initialize gpt2 tokenizer and model**

In [2]:
from google.colab import userdata
hugging_face_token = userdata.get('hugging_face_read_token')
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

**Loading a dataset**

In [3]:
%pip install datasets

In [4]:
from datasets import load_dataset
dataset_name = 'sst2'
ds = load_dataset(dataset_name)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [6]:
ds_train, ds_val = ds['train'], ds['validation']

**Tokenizing a Dataset**

In [7]:
def tokenize(batch):
    return tokenizer(batch['sentence'])

map_kwargs = {
    'batched':True,
    'batch_size':512,
    'remove_columns':['idx','sentence','label']
}

tokenized_dataset_train = ds_train.map(tokenize, **map_kwargs)
tokenized_dataset_val = ds_val.map(tokenize, **map_kwargs)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Filter out sentences shorter than 5 tokens

In [8]:
tokenized_dataset_train = tokenized_dataset_train.filter(lambda x: len(x['input_ids']) > 5)
tokenized_dataset_val = tokenized_dataset_val.filter(lambda x: len(x['input_ids']) > 5)

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

Filter:   0%|          | 0/872 [00:00<?, ? examples/s]

In [9]:
tokenized_dataset_train.set_format('torch')
tokenized_dataset_val.set_format('torch')

In [10]:
tokenizer.pad_token = tokenizer.eos_token

In [11]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm=False)

dataloader_params = {
    'batch_size':32,
    'collate_fn':data_collator
}

dataloader_train = DataLoader(tokenized_dataset_train, **dataloader_params)
dataloader_val = DataLoader(tokenized_dataset_val, **dataloader_params)

In [12]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
model.to(device)
num_epochs = 1

In [13]:
def validate(epoch):
    model.eval()
    total_loss = 0.0
    for i, batch in enumerate(dataloader_val):
        # iteration = epoch * len(dataloader_val) + i
        batch = batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
            loss = outputs.loss # Uses transformers.loss.loss_utils.ForCausalLMLoss for loss calculation
            total_loss += loss.item()
    print(f'val_loss at {epoch} epoch:', total_loss / len(dataloader_val))

Loss function code from [here](https://github.com/huggingface/transformers/blob/main/src/transformers/loss/loss_utils.py)

In [14]:
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(dataloader_train):
      batch = batch.to(device)
      outputs = model(**batch)
      loss = outputs.loss
      print(f'Loss: {loss.item()}')
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    validate(epoch+1)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Loss: 5.8966193199157715
Loss: 5.6709723472595215
Loss: 5.2181620597839355
Loss: 5.270534038543701
Loss: 5.447666168212891
Loss: 5.378504753112793
Loss: 5.0543670654296875
Loss: 4.892578125
Loss: 4.8366923332214355
Loss: 4.931551456451416
Loss: 4.852812767028809
Loss: 4.625549793243408
Loss: 4.828558921813965
Loss: 4.94966983795166
Loss: 4.503764629364014
Loss: 4.501881122589111
Loss: 4.548290252685547
Loss: 4.6265668869018555
Loss: 4.712891101837158
Loss: 4.640940189361572
Loss: 4.6615729331970215
Loss: 4.474776744842529
Loss: 4.44701623916626
Loss: 4.61978006362915
Loss: 4.507959842681885
Loss: 4.493252754211426
Loss: 4.615767002105713
Loss: 4.397085189819336
Loss: 4.482495307922363
Loss: 4.703412055969238
Loss: 4.553531646728516
Loss: 4.438678741455078
Loss: 4.27724027633667
Loss: 4.44836950302124
Loss: 4.46668815612793
Loss: 4.411080360412598
Loss: 4.428953170776367
Loss: 4.462832450866699
Loss: 4.295839309692383
Loss: 4.332491397857666
Loss: 4.660189151763916
Loss: 4.3371887207031

**Save the model and zip saved model**

In [15]:
model.save_pretrained('./sft_model_epoch_1')

In [16]:
model.from_pretrained('./sft_model_epoch_1')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [17]:
!zip -r sft_model_epoch_1.zip sft_model_epoch_1

  adding: sft_model_epoch_1/ (stored 0%)
  adding: sft_model_epoch_1/generation_config.json (deflated 24%)
  adding: sft_model_epoch_1/model.safetensors (deflated 7%)
  adding: sft_model_epoch_1/config.json (deflated 51%)
